In [14]:
import numpy as np
import netket as nk
# import functions as f1
from scripts import functions as f

In [15]:
from conf import *
from numba import njit

In [65]:
h = 1.0
V = 1.1

length = [12, 12]

sigmaz = np.array([[1, 0], [0, -1]])
sigmax = np.array([[0,1],[1,0]])
sigmay = np.array([[0,1],[-1,0]])

mszsz = np.kron(sigmaz, sigmaz)
mszsx = np.kron(sigmax, sigmax)
mszsy = np.kron(sigmay, sigmay)

potential_anti = (np.identity(4) + mszsz)/2
potential_ferro = (np.identity(4) - mszsz)/2

# hexagon = nk.machine.graph_hex(length = length)
hexagon = f.new_hex(np.array(length))


g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])

e, ec, ce, cec = hexagon.for_hamiltonian()

hi = nk.hilbert.Spin(s=0.5, graph=g)
op = nk.operator.DimerLocalOperator2(hi)

for edge, edge_color, pe, pec in zip(e, ec, ce, cec):

    l_op = -h*mszsx

    l_op = np.kron(np.identity(2), l_op)
    l_op = np.kron(l_op, np.identity(2))

    mat = []
    edge_ = []
    for p, c in zip(pe, pec):
        mat.append(np.kron(
            potential_ferro if c[0] == 1 else potential_anti,
            potential_ferro if c[1] == 1 else potential_anti,
        ))
        edge_.append(p[0].tolist() + p[1].tolist())

    op += nk.operator.DimerLocalOperator2(hi, l_op @ mat[0] + V * mat[0], edge_[0])
    op += nk.operator.DimerLocalOperator2(hi, l_op @ mat[1] + V * mat[1], edge_[1])

    
def dimer_flip(h = 1,length = [4, 2]):


    sigmaz = np.array([[1, 0], [0, -1]])
    sigmax = np.array([[0,1],[1,0]])
    sigmay = np.array([[0,1],[-1,0]])

    mszsz = np.kron(sigmaz, sigmaz)
    mszsx = np.kron(sigmax, sigmax)
    mszsy = np.kron(sigmay, sigmay)

    potential_anti = (np.identity(4) + mszsz)/2
    potential_ferro = (np.identity(4) - mszsz)/2

    # hexagon = nk.machine.graph_hex(length = length)
    hexagon = nk.machine.new_hex(np.array(length))


    g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])

    e, ec, ce, cec = hexagon.for_hamiltonian()

    hi = nk.hilbert.Spin(s=0.5, graph=g)
    op = nk.operator.DimerLocalOperator2(hi)

    for edge, edge_color, pe, pec in zip(e, ec, ce, cec):

        l_op = -h*mszsx
        
        l_op = np.kron(np.identity(2), l_op)
        l_op = np.kron(l_op, np.identity(2))
        
        mat = []
        edge_ = []
        for p, c in zip(pe, pec):
            mat.append(np.kron(
                potential_ferro if c[0] == 1 else potential_anti,
                potential_ferro if c[1] == 1 else potential_anti,
            ))
            edge_.append(p[0].tolist() + p[1].tolist())
        
        op += nk.operator.DimerLocalOperator2(hi, l_op @ mat[0], edge_[0])
        op += nk.operator.DimerLocalOperator2(hi, l_op @ mat[1], edge_[1])
    
    return op

In [66]:
g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)
op_transition2 = f.dimer_flip2(length = np.array(length))
op_transition1 = f.dimer_flip1(length = np.array(length))

In [67]:
alpha = 1
hex_ = nk.machine.new_hex(np.array(length))
ma = nk.machine.RbmDimer(hi,hex_, alpha = alpha, symmetry = True, reverse=True
                    ,use_hidden_bias = False, use_visible_bias = False, dtype=float)
ma.init_random_parameters(seed=1234)

2


In [62]:
ma._autom.shape

(100, 200)

In [63]:
n_chains = 1
kernel = 1
sa = nk.sampler.DimerMetropolisLocal(machine=ma, op=op_transition1, length = length, sweep_size = sweep_size, n_chains = n_chains, kernel = kernel)
P = sa.generate_samples(10000)

KeyboardInterrupt: 

In [49]:
%time ma.log_val(P.reshape(-1, 128))

CPU times: user 399 ms, sys: 8.26 ms, total: 407 ms
Wall time: 34.6 ms


array([1.55613336+0.j, 1.51333964+0.j, 1.48063711+0.j, ...,
       1.76727782+0.j, 1.50699975+0.j, 1.32587978+0.j])

In [68]:
kernel = 1
n_chains = 1
n_iter = 600
decay_factor = 1
sa = nk.sampler.DimerMetropolisLocal(machine=ma, op=op_transition1, length = length, sweep_size = sweep_size, n_chains = n_chains, kernel = kernel)
sr = nk.optimizer.SR(ma, diag_shift=0)
opt = nk.optimizer.Sgd(ma, learning_rate=0.05, decay_factor = decay_factor ,N = n_iter)
gs = nk.Vmc(
hamiltonian=op,
sampler=sa,
optimizer=opt,
n_samples=2000,
sr = sr,
n_discard = 300,
)

gs.run(n_iter=n_iter)

  0%|          | 0/600 [00:00<?, ?it/s]

discard =  300
No output specified (out=[apath|nk.logging.JsonLogger(...)]).Running the optimization but not saving the output.
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. 

  0%|          | 1/600 [00:20<3:23:05, 20.34s/it, Energy=4.131+0.000j ± 0.017 [σ²=0.455]]

[ 1.  1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1.  1.
  1.  1.  1.  1.  1.  1. -1. -1.  1.  1. -1.  1.  1.  1. -1. -1. -1. -1.
 -1.  1.  1. -1.  1.  1.  1. -1. -1. -1. -1. -1.  1.  1. -1. -1. -1.  1.
 -1. -1. -1. -1.  1.  1.  1.  1. -1. -1.  1.  1. -1. -1. -1. -1.  1.  1.
  1.  1.  1. -1. -1.  1.  1. -1.  1. -1. -1.  1.  1.  1.  1. -1.  1.  1.
 -1.  1. -1. -1.  1.  1.  1.  1. -1. -1. -1.  1.  1.  1.  1. -1. -1.  1.
  1.  1. -1. -1.  1.  1.  1.  1. -1. -1.  1.  1. -1.  1.  1. -1. -1. -1.
 -1.  1.  1. -1. -1. -1. -1.  1.  1. -1. -1. -1. -1.  1. -1. -1. -1. -1.
  1.  1. -1. -1. -1. -1.  1.  1.  1. -1.  1.  1.  1. -1. -1. -1. -1.  1.
  1.  1. -1.  1.  1.  1. -1. -1. -1. -1.  1.  1. -1.  1.  1.  1.  1.  1.
 -1. -1. -1. -1.  1. -1.  1.  1.  1.  1.  1.  1.  1. -1. -1. -1. -1. -1.
 -1.  1. -1. -1. -1.  1.  1. -1. -1. -1. -1. -1.  1. -1. -1. -1. -1.  1.
 -1. -1. -1. -1. -1. -1. -1. -1.  1.  1.  1.  1. -1. -1. -1. -1. -1. -1.
 -1. -1.  1.  1.  1. -1. -1. -1. -1. -1.  1. -1.  1

  0%|          | 2/600 [00:43<3:38:33, 21.93s/it, Energy=4.117+0.000j ± 0.018 [σ²=0.374]]

...

[-1. -1. -1. -1. -1.  1. -1.  1.  1. -1. -1.  1. -1. -1. -1. -1.  1. -1.
  1.  1. -1. -1.  1. -1.  1. -1. -1. -1. -1. -1. -1.  1.  1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1.  1.  1. -1. -1.  1.  1.  1.  1. -1. -1. -1.
  1.  1. -1. -1.  1.  1.  1.  1. -1. -1. -1.  1.  1. -1. -1.  1.  1.  1.
  1.  1.  1.  1.  1.  1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
 -1. -1.  1.  1.  1.  1. -1.  1. -1. -1. -1.  1.  1. -1. -1.  1. -1. -1.
  1.  1. -1. -1.  1.  1. -1. -1.  1. -1. -1. -1. -1.  1.  1. -1. -1.  1.
  1. -1. -1. -1. -1. -1.  1.  1. -1. -1.  1.  1.  1. -1. -1. -1. -1. -1.
 -1.  1.  1. -1. -1. -1. -1. -1.  1. -1.  1. -1. -1.  1.  1. -1. -1. -1.
 -1.  1. -1.  1. -1. -1.  1.  1. -1. -1.  1.  1.  1.  1.  1.  1.  1. -1.
  1. -1. -1. -1.  1.  1.  1.  1.  1.  1.  1. -1. -1. -1.  1. -1. -1.  1.
 -1. -1. -1. -1. -1. -1. -1. -1.  1. -1. -1.  1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1.  1.  1.  1.  1.  1. -1. -1. -1. -1. -1. -1.  1.  1.  1.
  1.  1. -1. -1. -1. -1.  1. -1.  1.  1. -1. -1. -1

 19%|█▉        | 116/600 [39:50<2:48:43, 20.92s/it, Energy=3.8849+0.0000j ± 0.0088 [σ²=0.0874]]

[-1.  1. -1. -1.  1.  1.  1. -1. -1.  1. -1. -1.  1. -1. -1. -1.  1.  1.
 -1. -1.  1.  1. -1. -1. -1. -1. -1. -1. -1.  1.  1. -1. -1.  1.  1. -1.
 -1. -1. -1. -1. -1.  1.  1. -1.  1.  1.  1. -1.  1.  1.  1.  1.  1.  1.
 -1. -1. -1.  1. -1. -1.  1.  1.  1.  1.  1.  1. -1. -1.  1.  1. -1.  1.
  1.  1.  1.  1. -1.  1.  1. -1. -1.  1.  1.  1.  1.  1.  1.  1. -1.  1.
  1. -1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1. -1. -1. -1. -1. -1.  1.
  1. -1. -1.  1.  1. -1. -1. -1. -1. -1.  1.  1. -1. -1.  1.  1. -1. -1.
  1. -1. -1. -1. -1. -1. -1.  1.  1.  1. -1.  1. -1. -1. -1. -1. -1. -1.
  1.  1. -1.  1.  1.  1.  1.  1.  1. -1.  1.  1.  1. -1.  1.  1.  1.  1.
  1.  1.  1. -1.  1.  1. -1. -1. -1.  1. -1. -1. -1. -1. -1. -1. -1.  1.
 -1. -1. -1.  1. -1. -1. -1. -1. -1. -1. -1.  1.  1.  1.  1.  1.  1. -1.
 -1. -1. -1. -1.  1.  1.  1.  1.  1.  1.  1. -1. -1. -1. -1.  1.  1.  1.
 -1.  1. -1. -1. -1. -1.  1.  1.  1.  1. -1.  1.  1.  1. -1. -1. -1.  1.
  1.  1.  1. -1.  1. -1. -1.  1.  1. -1.  1.  1.  1

 19%|█▉        | 116/600 [39:53<2:46:25, 20.63s/it, Energy=3.8849+0.0000j ± 0.0088 [σ²=0.0874]]


KeyboardInterrupt: 

In [ ]:
kernel = 2
n_chains = 16
n_iter = 100
decay_factor = 'sigmoid decay'
sa = nk.sampler.DimerMetropolisLocal(machine=ma, op=op_transition1, length = length, sweep_size = sweep_size, n_chains = n_chains, kernel = kernel)
sr = nk.optimizer.SR(ma, diag_shift=0)
opt = nk.optimizer.Sgd(ma, learning_rate=0.05, decay_factor = decay_factor ,N = n_iter)
gs = nk.Vmc(
hamiltonian=op,
sampler=sa,
optimizer=opt,
n_samples=20000,
sr = sr,
n_discard = 300,
)

gs.run(n_iter=n_iter)

In [ ]:
sample_

In [ ]:
@njit
def foo(x,y):
    print((x//y).astype(np.int64))
    return x

In [ ]:
x = np.random.randint(0,100, 10)
y = np.random.randint(1,10, 10)

In [ ]:
foo(x,y)

In [ ]:

name = 'h={}V={}l={}'.format(h, V, length)
n_chains = 10
sweep_size = 70
n_samples = 100000
kernel = 1


g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)

op = f.dimer_hamiltonian(h, V,np.array(length))
op_transition = f.dimer_flip2(length = np.array(length))
hex_ = nk.machine.new_hex(np.array(length))


ma = nk.machine.RbmDimer(hi, hex_, alpha = alpha, symmetry = True
                    ,use_hidden_bias = False, use_visible_bias = False, dtype=float)

ma.load('save/ma/'+name)


sa = nk.sampler.DimerMetropolisLocal(machine=ma, op=op_transition, length = length, n_chains=n_chains, sweep_size = 70, kernel = kernel)
sa.generate_samples(500) # discard the begginings of metropolis sampling.
print('discard samples')
samples_state = sa.generate_samples(int(n_samples / n_chains))
samples_state = samples_state.reshape(-1, ma.hilbert.size)

print('prepared initial samples')




In [ ]:
np.int64(10)

In [ ]:
hexagon.is_dimer_basis(samples_state)
sections = np.ones(n_samples, dtype=np.int)

In [ ]:
# %%time
for _ in range(1000):
    op_transition2.get_conn_flattened(samples_state[:100], sections[:100])

In [ ]:
local_states = op._local_states
basis = op._basis

In [ ]:
samples_state

In [ ]:
((samples_state[:, op._acting_on[0]] + 1) / 2 * basis).sum(axis=1)

In [ ]:
def dot_1(x, y):
    return (x * y).sum(axis=1)

@njit
def dot_numba(x,z):
    y = np.zeros(x.shape[0])
    for i in range(x.shape[1]):
        y += x[:,i] * z[i]
    return y 

In [ ]:
# %%timeit 
X = (np.random.choice([-1,1], (10000,4)) + 1)/2
i = dot_1(X, basis)

In [ ]:
op._n_conns[i.astype(np.int)]

In [ ]:
i

In [ ]:
conn = op._n_conns

In [ ]:
# %%timeit 
X = np.random.randint(1,2, (10000,4))
dot_numba(X, basis)

In [ ]:
%%timeit 
a = 0
for k in range(acting_size):
    a += (
        _np.searchsorted(local_states, x_i[acting_size- k - 1])
        * basis[k]
    )

In [ ]:
@njit
def dot(x, acting_on):
    
    xx = x[:, acting_on]
    
    return xx

In [ ]:
dot(samples_state, op._acting_on[0])